# Importing Libraries

In [71]:
import time
import pandas as pd
import polars as pl
import plotly.graph_objects as go

## Settings and Functions

In [72]:
def plot_comparison(sizes, pandas, polars, title):
    # Convertendo os tempos de leitura para texto com 2 casas decimais
    pandas_times_text = [f"{time:.2f}" for time in pandas]
    polars_times_text = [f"{time:.2f}" for time in polars]

    # Criar um gráfico de barras com Plotly
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=[str(size) for size in sizes],
        y=pandas,
        name='Pandas',
        marker_color='blue',
        text=pandas_times_text,
        textposition='auto'
    ))

    fig.add_trace(go.Bar(
        x=[str(size) for size in sizes],
        y=polars,
        name='Polars',
        marker_color='green',
        text=polars_times_text,
        textposition='auto'
    ))

    fig.update_layout(
        title=f'Comparação de Tempo de {title}: Pandas vs Polars (csv)',
        xaxis_title='Tamanho do Arquivo',
        yaxis_title=f'Tempo de {title} (s)',
        barmode='group'
    )

    fig.show()

# Defining the size of files to be test

In [73]:
# Tamanhos dos arquivos
csv_sizes = [50000, 100000, 250000, 500000, 1000000, 2500000, 5000000]
xlsx_sizes = [50000, 100000, 250000, 500000, 1000000]

# Testing the read method

## csv file

In [74]:
# Funções de benchmarking para Pandas
def benchmark_pandas_read_csv(file_path, size):
    start = time.time()
    globals()[f'df_mock_data_{size}_pandas_csv'] = pd.read_csv(file_path)
    end = time.time()
    return end - start


# Funções de benchmarking para Polars
def benchmark_polars_read_csv(file_path, size):
    start = time.time()
    globals()[f'df_mock_data_{size}_polars_csv'] = pl.read_csv(file_path)
    end = time.time()
    return end - start

In [75]:
# Listas para armazenar os tempos de leitura
pandas_read_csv_times = []
polars_read_csv_times = []

# Comparação de desempenho
for size in csv_sizes:
    csv_path = f"mock_data/mock_data_{size}.csv"
    
    # Pandas
    pandas_read_csv_time = benchmark_pandas_read_csv(csv_path, size)
    pandas_read_csv_times.append(pandas_read_csv_time)
    
    # Polars
    polars_read_csv_time = benchmark_polars_read_csv(csv_path, size)
    polars_read_csv_times.append(polars_read_csv_time)

# Formatando os tempos para ter 2 casas decimais
pandas_read_csv_times_text = [f"{time:.2f}" for time in pandas_read_csv_times]
polars_read_csv_times_text = [f"{time:.2f}" for time in polars_read_csv_times]

plot_comparison(csv_sizes, pandas_read_csv_times, polars_read_csv_times, 'Leitura')

In [76]:
# Criando um DataFrame
df_read_csv = pd.DataFrame({
    'CSV_Size': csv_sizes,
    'Pandas_Read_Time': pandas_read_csv_times,
    'Polars_Read_Time': polars_read_csv_times
})

# Calculando a porcentagem de melhoria
df_read_csv['Improvement (%)'] = (df_read_csv['Pandas_Read_Time'] - df_read_csv['Polars_Read_Time']) / df_read_csv['Pandas_Read_Time'] * 100

df_read_csv

,CSV_Size,Pandas_Read_Time,Polars_Read_Time,Improvement (%)
0,50000,0.467231,0.058339,87.513963
1,100000,0.467451,0.073997,84.170214
2,250000,1.268631,0.079789,93.710610
3,500000,2.391207,0.122504,94.876916
4,1000000,3.345535,0.269642,91.940232
5,2500000,10.464175,0.597520,94.289847
6,5000000,25.410099,8.003066,68.504391


## xlsx file

In [77]:
# Funções de benchmarking para Pandas
def benchmark_pandas_read_xlsx(file_path, size):
    start = time.time()
    globals()[f'df_mock_data_{size}_pandas_xlsx'] = pd.read_excel(file_path)
    end = time.time()
    return end - start


# Funções de benchmarking para Polars
def benchmark_polars_read_xlsx(file_path, size):
    start = time.time()
    globals()[f'df_mock_data_{size}_polars_xlsx'] = pl.read_excel(file_path)
    end = time.time()
    return end - start

In [79]:
pandas_read_xlsx_times = []
polars_read_xlsx_times = []

# Comparação de desempenho
for size in xlsx_sizes:
    xlsx_path = f"mock_data/mock_data_{size}.xlsx"
    
    # Pandas
    pandas_read_xlsx_time = benchmark_pandas_read_xlsx(xlsx_path, size)
    pandas_read_xlsx_times.append(pandas_read_xlsx_time)
    
    # Polars
    polars_read_xlsx_time = benchmark_polars_read_xlsx(xlsx_path, size)
    polars_read_xlsx_times.append(polars_read_xlsx_time)

plot_comparison(xlsx_sizes, pandas_read_xlsx_times, polars_read_xlsx_times, 'Leitura')

In [80]:
# Criando um DataFrame
df_read_xlsx = pd.DataFrame({
    'XLSX_Size': xlsx_sizes,
    'Pandas_Read_Time': pandas_read_xlsx_times,
    'Polars_Read_Time': polars_read_xlsx_times
})

# Calculando a porcentagem de melhoria
df_read_xlsx['Improvement (%)'] = (df_read_xlsx['Pandas_Read_Time'] - df_read_xlsx['Polars_Read_Time']) / df_read_xlsx['Pandas_Read_Time'] * 100

df_read_xlsx

,XLSX_Size,Pandas_Read_Time,Polars_Read_Time,Improvement (%)
0,50000,6.969049,4.212656,39.551929
1,100000,11.440848,6.977215,39.014881
2,250000,29.379991,14.330857,51.222390
3,500000,61.634115,35.296012,42.732994
4,1000000,139.010938,77.499611,44.249271


# Testing the filter (loc) method

## csv file

In [101]:
# Funções de benchmarking para Pandas
def benchmark_pandas_filter_csv(size):
    start = time.time()
    df = globals()[f'df_mock_data_{size}_pandas_csv']
    df_filtered = df.loc[df['B'] > 0.5]
    end = time.time()
    return end - start


# Funções de benchmarking para Polars
def benchmark_polars_filter_csv(size):
    start = time.time()
    df = globals()[f'df_mock_data_{size}_polars_csv']
    df_filtered = df.filter(df['B'] > 0.5)
    end = time.time()
    return end - start

In [102]:
pandas_filter_csv_times = []
polars_filter_csv_times = []

# Comparação de desempenho
for size in csv_sizes:
    
    # Pandas
    pandas_filter_csv_time = benchmark_pandas_filter_csv(size)
    pandas_filter_csv_times.append(pandas_filter_csv_time)
    
    # Polars
    polars_filter_csv_time = benchmark_polars_filter_csv(size)
    polars_filter_csv_times.append(polars_filter_csv_time)

# Formatando os tempos para ter 2 casas decimais
pandas_filter_csv_times_text = [f"{time:.2f}" for time in pandas_filter_csv_times]
polars_filter_csv_times_text = [f"{time:.2f}" for time in polars_filter_csv_times]

plot_comparison(csv_sizes, pandas_filter_csv_times, polars_filter_csv_times, 'Filtro')

In [103]:
# Criando um DataFrame
df_filter_csv = pd.DataFrame({
    'CSV_Size': csv_sizes,
    'Pandas_Filter_Time': pandas_filter_csv_times,
    'Polars_Filter_Time': polars_filter_csv_times
})

# Calculando a porcentagem de melhoria
df_filter_csv['Improvement (%)'] = (df_filter_csv['Pandas_Filter_Time'] - df_filter_csv['Polars_Filter_Time']) / df_filter_csv['Pandas_Filter_Time'] * 100

df_filter_csv

,CSV_Size,Pandas_Filter_Time,Polars_Filter_Time,Improvement (%)
0,50000,0.063746,0.032307,49.319106
1,100000,0.077778,0.027617,64.491774
2,250000,0.144632,0.066445,54.059311
3,500000,0.340183,0.115693,65.991045
4,1000000,1.265689,0.257863,79.626665
5,2500000,4.837497,2.350289,51.415189
6,5000000,12.425378,5.453388,56.110886


## xlsx file

In [104]:
# Funções de benchmarking para Pandas
def benchmark_pandas_filter_xlsx(size):
    start = time.time()
    df = globals()[f'df_mock_data_{size}_pandas_xlsx']
    df_filtered = df.loc[df['B'] > 0.5]
    end = time.time()
    return end - start


# Funções de benchmarking para Polars
def benchmark_polars_filter_xlsx(size):
    start = time.time()
    df = globals()[f'df_mock_data_{size}_polars_xlsx']
    df_filtered = df.filter(df['B'] > 0.5)
    end = time.time()
    return end - start

In [105]:
pandas_filter_xlsx_times = []
polars_filter_xlsx_times = []

# Comparação de desempenho
for size in xlsx_sizes:
    
    # Pandas
    pandas_filter_xlsx_time = benchmark_pandas_filter_xlsx(size)
    pandas_filter_xlsx_times.append(pandas_filter_xlsx_time)
    
    # Polars
    polars_filter_xlsx_time = benchmark_polars_filter_xlsx(size)
    polars_filter_xlsx_times.append(polars_filter_xlsx_time)

# Formatando os tempos para ter 2 casas decimais
pandas_filter_xlsx_times_text = [f"{time:.2f}" for time in pandas_filter_xlsx_times]
polars_filter_xlsx_times_text = [f"{time:.2f}" for time in polars_filter_xlsx_times]

plot_comparison(xlsx_sizes, pandas_filter_xlsx_times, polars_filter_xlsx_times, 'Filtro')

In [106]:
# Criando um DataFrame
df_filter_xlsx = pd.DataFrame({
    'XLSX_Size': xlsx_sizes,
    'Pandas_Filter_Time': pandas_filter_xlsx_times,
    'Polars_Filter_Time': polars_filter_xlsx_times
})

# Calculando a porcentagem de melhoria
df_filter_xlsx['Improvement (%)'] = (df_filter_xlsx['Pandas_Filter_Time'] - df_filter_xlsx['Polars_Filter_Time']) / df_filter_xlsx['Pandas_Filter_Time'] * 100

df_filter_xlsx

,XLSX_Size,Pandas_Filter_Time,Polars_Filter_Time,Improvement (%)
0,50000,0.195359,0.059701,69.440342
1,100000,0.107003,0.026887,74.872383
2,250000,0.354185,0.071876,79.706724
3,500000,0.438984,0.104149,76.275029
4,1000000,1.737552,0.472653,72.797781


# Testing the calculation method

## csv file

In [90]:
# Funções de benchmarking para Pandas
def benchmark_pandas_calc_csv(df, size):
    start = time.time()
    df = globals()[f'df_mock_data_{size}_pandas_csv']
    df['G'] = df['B'] * df['C']
    end = time.time()
    return end - start


# Funções de benchmarking para Polars
def benchmark_polars_calc_csv(df, size):
    start = time.time()
    df = globals()[f'df_mock_data_{size}_polars_csv']
    new_col = (df['B'] * df['C']).alias('G')
    df = df.hstack([new_col])
    end = time.time()
    return end - start

In [91]:
pandas_calc_csv_times = []
polars_calc_csv_times = []

# Comparação de desempenho
for size in csv_sizes:
    csv_path = f"mock_data/mock_data_{size}.csv"
    
    # Pandas
    pandas_calc_csv_time = benchmark_pandas_calc_csv(csv_path, size)
    pandas_calc_csv_times.append(pandas_calc_csv_time)
    
    # Polars
    polars_calc_csv_time = benchmark_polars_calc_csv(csv_path, size)
    polars_calc_csv_times.append(polars_calc_csv_time)

# Formatando os tempos para ter 2 casas decimais
pandas_calc_csv_times_text = [f"{time:.2f}" for time in pandas_calc_csv_times]
polars_calc_csv_times_text = [f"{time:.2f}" for time in polars_calc_csv_times]

plot_comparison(csv_sizes, pandas_calc_csv_times, polars_calc_csv_times, 'Filtro')

In [92]:
# Criando um DataFrame
df_calc_csv = pd.DataFrame({
    'CSV_Size': csv_sizes,
    'Pandas_Calc_Time': pandas_calc_csv_times,
    'Polars_Calc_Time': polars_calc_csv_times
})

# Calculando a porcentagem de melhoria
df_calc_csv['Improvement (%)'] = (df_calc_csv['Pandas_Calc_Time'] - df_calc_csv['Polars_Calc_Time']) / df_calc_csv['Pandas_Calc_Time'] * 100

df_calc_csv

,CSV_Size,Pandas_Calc_Time,Polars_Calc_Time,Improvement (%)
0,50000,0.010566,0.002002,81.053819
1,100000,0.004994,0.001004,79.891149
2,250000,0.006053,0.001989,67.139312
3,500000,0.025264,0.002988,88.171678
4,1000000,0.026356,0.009639,63.429703
5,2500000,0.057770,0.063981,-10.750412
6,5000000,0.063253,0.076557,-21.033845


## xlsx file

In [93]:
# Funções de benchmarking para Pandas
def benchmark_pandas_calc_xlsx(df, size):
    start = time.time()
    df = globals()[f'df_mock_data_{size}_pandas_xlsx']
    df_filtered = df.loc[df['B'] > 0.5]
    end = time.time()
    return end - start


# Funções de benchmarking para Polars
def benchmark_polars_calc_xlsx(df, size):
    start = time.time()
    df = globals()[f'df_mock_data_{size}_polars_xlsx']
    df_filtered = df.filter(df['B'] > 0.5)
    end = time.time()
    return end - start

In [94]:
pandas_calc_xlsx_times = []
polars_calc_xlsx_times = []

# Comparação de desempenho
for size in xlsx_sizes:
    xlsx_path = f"mock_data/mock_data_{size}.xlsx"
    
    # Pandas
    pandas_calc_xlsx_time = benchmark_pandas_calc_xlsx(xlsx_path, size)
    pandas_calc_xlsx_times.append(pandas_calc_xlsx_time)
    
    # Polars
    polars_calc_xlsx_time = benchmark_polars_calc_xlsx(xlsx_path, size)
    polars_calc_xlsx_times.append(polars_calc_xlsx_time)

# Formatando os tempos para ter 2 casas decimais
pandas_calc_xlsx_times_text = [f"{time:.2f}" for time in pandas_calc_xlsx_times]
polars_calc_xlsx_times_text = [f"{time:.2f}" for time in polars_calc_xlsx_times]

plot_comparison(xlsx_sizes, pandas_calc_xlsx_times, polars_calc_xlsx_times, 'Filtro')

In [95]:
# Criando um DataFrame
df_calc_xlsx = pd.DataFrame({
    'XLSX_Size': xlsx_sizes,
    'Pandas_Calc_Time': pandas_calc_xlsx_times,
    'Polars_Calc_Time': polars_calc_xlsx_times
})

# Calculando a porcentagem de melhoria
df_calc_xlsx['Improvement (%)'] = (df_calc_xlsx['Pandas_Calc_Time'] - df_calc_xlsx['Polars_Calc_Time']) / df_calc_xlsx['Pandas_Calc_Time'] * 100

df_calc_xlsx

,XLSX_Size,Pandas_Calc_Time,Polars_Calc_Time,Improvement (%)
0,50000,0.132506,0.048350,63.510805
1,100000,0.140163,0.035285,74.825434
2,250000,0.785073,0.124209,84.178639
3,500000,1.579608,0.343925,78.227184
4,1000000,2.697108,1.391413,48.410920


# Testing the iteration (for) method

In [ ]:
# Criando um DataFrame
df_calc_xlsx = pd.DataFrame({
    'XLSX_Size': xlsx_sizes,
    'Pandas_Calc_Time': pandas_calc_xlsx_times,
    'Polars_Calc_Time': polars_calc_xlsx_times
})

# Calculando a porcentagem de melhoria
df_calc_xlsx['Improvement (%)'] = (df_calc_xlsx['Pandas_Calc_Time'] - df_calc_xlsx['Polars_Calc_Time']) / df_calc_xlsx['Pandas_Calc_Time'] * 100

df_calc_xlsx

,XLSX_Size,Pandas_Calc_Time,Polars_Calc_Time,Improvement (%)
0,50000,0.132506,0.048350,63.510805
1,100000,0.140163,0.035285,74.825434
2,250000,0.785073,0.124209,84.178639
3,500000,1.579608,0.343925,78.227184
4,1000000,2.697108,1.391413,48.410920
